# LLM 프롬프팅 실습

이 실습은 Anthrophic의 [공식 프롬프트 엔지니어링 가이드](https://docs.anthropic.com/ko/docs/build-with-claude/prompt-engineering/overview)의 내용을 기반으로 SageMaker 노트북 환경에서 실습해볼 수 있도록 만들어졌습니다.

**목차**

0. [준비](#0.-준비)
1. [명확하고 직접적이며 상세하게 설명하기](#1.-명확하고-직접적이며-상세하게-설명하기)
2. [예시를 사용하여 Claude의 행동 안내하기 (multishot prompting)](#2.-예시를-사용하여-Claude의-행동-안내하기-(multishot-prompting))
3. [Claude가 생각하게 하기(chain of thought prompting)로 성능 향상시키기](#3.-Claude가-생각하게-하기(chain-of-thought-prompting)로-성능-향상시키기)
4. [프롬프트 구조화를 위해 XML 태그 사용하기](#4.-프롬프트-구조화를-위해-XML-태그-사용하기)
5. [시스템 프롬프트로 Claude에게 역할 부여하기](#5.-시스템-프롬프트로-Claude에게-역할-부여하기)
6. [출력 제어를 위해 Claude의 응답 미리 채우기](#6.-출력-제어를-위해-Claude의-응답-미리-채우기)

## 0. 준비

Amazon Bedrock의 Claude를 호출하기 위한 상수와 함수를 선언합니다.

In [ ]:
# MODEL_ID = "anthropic.claude-v2:1"
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
# MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
BEDROCK_REGION = "us-west-2"

In [ ]:
import boto3
import json

def get_bedrock_client():
    """
    Amazon Bedrock 클라이언트를 생성하고 반환합니다.
    """
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        region_name=BEDROCK_REGION
    )
    return bedrock_runtime

def invoke_claude(prompt, max_tokens=500):
    """
    Claude 3 Sonnet 모델을 호출하여 응답을 생성합니다.

    :param prompt: 모델에게 전달할 프롬프트
    :param max_tokens: 생성할 최대 토큰 수
    :return: 모델의 응답
    """
    bedrock_client = get_bedrock_client()
    
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    })
    
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=body
    )
    
    response_body = json.loads(response.get('body').read())
    return response_body['content'][0]['text']

In [4]:
%%html
<style>
.rendered_html { text-align: left !important; }
.rendered_html table td { text-align: left !important; }
.rendered_html table th { text-align: left !important; }
</style>

## 1. 명확하고 직접적이며 상세하게 설명하기

Claude와 상호 작용할 때는 Claude를 뛰어나지만 매우 새로운 직원(기억상실증이 있는)으로 생각하고 명시적인 지침을 제공해야 합니다. 다른 신입 직원과 마찬가지로 Claude는 귀하의 규범, 스타일, 지침 또는 선호하는 작업 방식에 대한 맥락을 가지고 있지 않습니다. 원하는 바를 정확하게 설명할수록 Claude의 응답은 더 좋아질 것입니다.

### 예시

명확하지 않은 프롬프트 예시에서 Claude는 여전히 고객 이름을 남기는 등의 실수를 합니다.

역할|명확하지 않은 프롬프트|명확한 프롬프트
---|---|---
사용자 | 이 고객 피드백 메시지에서 모든 개인 식별 정보를 제거해 주세요: {{FEEDBACK_DATA}} | 귀하의 작업은 분기별 검토를 위해 고객 피드백을 익명화하는 것입니다.<br><br>지침:<br>1. 모든 고객 이름을 "CUSTOMER_[ID]"로 바꾸세요(예: "Jane Doe" → "CUSTOMER_001").<br>2. 이메일 주소를 "EMAIL_[ID]@example.com"으로 바꾸세요.<br>3. 전화번호를 "PHONE_[ID]"로 가리세요.<br>4. 메시지에 특정 제품(예: "AcmeCloud")이 언급되면 그대로 두세요.<br>5. PII가 발견되지 않으면 메시지를 그대로 복사하세요.<br>6. 처리된 메시지만 "---"로 구분하여 출력하세요.<br><br>처리할 데이터: {{FEEDBACK_DATA}}
Assistant (Claude의 응답)|익명화된 고객 피드백 메시지는 다음과 같습니다:<br>1. 저는 귀사의 소프트웨어를 몇 년 동안 사용해 왔고, 정말 좋았습니다. 그런데 최근에 AcmeCloud가 자주 다운되고 있어요. 고쳐주세요!<br>2. XXX, 귀하의 고객 지원 담당자는 정말 도움이 되었습니다. 그는 전체 프로세스를 안내해 주었어요. 설문 조사를 보내고 싶으시면 제 이메일은 XXX입니다.<br>3. 저는 John Smith이고, 만족스럽지 않습니다. XXX에 연락하려고 2시간 동안 대기 중이에요. 이건 용납할 수 없어요! | CUSTOMER_001: 저는 귀사의 소프트웨어를 몇 년 동안 사용해 왔고, 정말 좋았습니다. 그런데 최근에 AcmeCloud가 자주 다운되고 있어요. 고쳐주세요!\n---\nCUSTOMER_002: CUSTOMER_SERVICE_001, 귀하의 고객 지원 담당자는 정말 도움이 되었습니다. 그는 전체 프로세스를 안내해 주었어요. 설문 조사를 보내고 싶으시면 제 이메일은 EMAIL_002@example.com입니다.\n---\nCUSTOMER_003: 저는 CUSTOMER_003이고, 만족스럽지 않습니다. PHONE_001에 연락하려고 2시간 동안 대기 중이에요. 이건 용납할 수 없어요!

In [ ]:
FEEDBACK_DATA = """
1. 저는 귀사의 소프트웨어를 몇 년 동안 사용해 왔고, 정말 좋았습니다. 그런데 최근에 AcmeCloud가 자주 다운되고 있어요. 고쳐주세요!
2. Steave, 귀하의 고객 지원 담당자는 정말 도움이 되었습니다. 그는 전체 프로세스를 안내해 주었어요. 설문 조사를 보내고 싶으시면 제 이메일은 smith@abc.com입니다.
3. 저는 John Smith이고, 만족스럽지 않습니다. 123-4567에 연락하려고 2시간 동안 대기 중이에요. 이건 용납할 수 없어요!
"""

prompt_01 = f"""
이 고객 피드백 메시지에서 모든 개인 식별 정보를 제거해 주세요: {FEEDBACK_DATA}
"""

# prompt_01 = f"""
# 귀하의 작업은 분기별 검토를 위해 고객 피드백을 익명화하는 것입니다.

# 지침:
# 1. 모든 고객 이름을 "CUSTOMER_[ID]"로 바꾸세요(예: "Jane Doe" → "CUSTOMER_001").
# 2. 이메일 주소를 "EMAIL_[ID]@example.com"으로 바꾸세요.
# 3. 전화번호를 "PHONE_[ID]"로 가리세요.
# 4. 메시지에 특정 제품(예: "AcmeCloud")이 언급되면 그대로 두세요.
# 5. PII가 발견되지 않으면 메시지를 그대로 복사하세요.
# 6. 처리된 메시지만 "---"로 구분하여 출력하세요.

# 처리할 데이터: {FEEDBACK_DATA}
# """

answer_01 = invoke_claude(prompt_01)
print(f"\n{answer_01}\n")

## 2. 예시를 사용하여 Claude의 행동 안내하기 (multishot prompting)

예시는 Claude가 정확히 필요한 것을 생성하도록 하는 비밀 무기입니다. 프롬프트에 잘 만들어진 몇 가지 예시를 제공함으로써 Claude의 출력물의 정확성, 일관성 및 품질을 크게 향상시킬 수 있습니다. Few-shot 또는 multishot prompting으로 알려진 이 기술은 구조화된 출력물이나 특정 형식을 준수해야 하는 작업에 특히 효과적입니다.

### 왜 예시를 사용할까요?

- 정확성: 예시는 지시 사항에 대한 오해를 줄입니다.
- 일관성: 예시는 통일된 구조와 스타일을 강제합니다.
- 성능: 잘 선택된 예시는 Claude가 복잡한 작업을 처리하는 능력을 향상시킵니다.


| 역할 | 예시 없음 | 예시 포함 |
|------|-----------|-----------|
| 사용자 | 이 고객 피드백을 분석하고 문제를 분류하세요. 다음 카테고리를 사용하세요: UI/UX, 성능, 기능 요청, 통합, 가격 책정, 기타. 또한 감정(긍정/중립/부정)과 우선순위(높음/중간/낮음)를 평가하세요. <br><br> 피드백은 다음과 같습니다: <br>{{FEEDBACK}} | 우리 CS 팀은 비정형 피드백으로 인해 압도되고 있습니다. 당신의 임무는 피드백을 분석하고 우리 제품 및 엔지니어링 팀을 위해 문제를 분류하는 것입니다. 다음 카테고리를 사용하세요: UI/UX, 성능, 기능 요청, 통합, 가격 책정, 기타. 또한 감정(긍정/중립/부정)과 우선순위(높음/중간/낮음)를 평가하세요. 다음은 예시입니다: <br><br> `<example>` 입력: 새 대시보드가 엉망이에요! 로딩하는 데 시간이 오래 걸리고 내보내기 버튼을 찾을 수 없어요. 당장 고쳐주세요! <br>카테고리: UI/UX, 성능 <br>감정: 부정 <br>우선순위: 높음`</example>` <br><br> 이제 이 피드백을 분석하세요: {{FEEDBACK}} |
| Assistant (Claude의 응답) | 고객 피드백 분석 결과는 다음과 같습니다:<br><br>문제: UI/UX<br>감정: 부정<br>우선순위: 높음<br>고객은 새로운 대시보드 디자인에 좌절감을 느끼고 있으며, 이를 어수선하고 로딩이 느리다고 생각합니다. 특히 누락된 "내보내기" 버튼을 지적하는데, 이는 중요한 기능일 가능성이 높습니다. 이 문제는 높은 우선순위로 해결되어야 합니다.<br><br>문제: 통합<br>감정: 긍정<br>우선순위: 중간<br>고객은 기존 Salesforce 통합에 만족하지만 Hubspot에 대한 지원도 보고 싶어 합니다. 이는 향후 제품 업데이트를 위해 고려할 수 있는 기능 요청입니다.<br><br>문제: 가격 책정<br>감정: 부정<br>우선순위: 높음<br>고객은 특히 기본 기능에 대한 가격이 터무니없다고 생각합니다. 경쟁사로 전환하겠다고 위협하고 있으므로 이는 검토가 필요한 높은 우선순위 문제입니다.<br><br>문제: 성능<br>감정: 부정<br>우선순위: 높음<br>고객은 모바일 앱에서 대용량 파일 업로드 시 지속적으로 충돌이 발생하는 심각한 문제를 겪고 있습니다. 이는 고객이 제품을 효과적으로 사용하는 데 영향을 미치는 중대한 문제이므로 높은 우선순위로 해결해야 합니다.<br><br>문제: UI/UX<br>감정: 긍정<br>우선순위: 낮음<br>고객은 전반적으로 제품에 만족하며, 특히 직관적인 UI와 훌륭한 고객 지원을 호평합니다. 이는 즉각적인 조치가 필요하지 않은 긍정적인 피드백입니다. | 고객 피드백 분석 결과는 다음과 같습니다:<br><br>1. 새 대시보드가 엉망이에요! 로딩하는 데 시간이 오래 걸리고 내보내기 버튼을 찾을 수 없어요. 당장 고쳐주세요!<br>카테고리: UI/UX, 성능<br>감정: 부정<br>우선순위: 높음<br><br>2. Salesforce 통합이 마음에 들어요! 하지만 Hubspot도 추가해 주시면 좋겠어요.<br>카테고리: 통합, 기능 요청<br>감정: 긍정<br>우선순위: 중간<br><br>3. 가격이 말도 안 돼요! 기본 기능에 월 $99라고? 경쟁사 X로 갈아탈 거예요.<br>카테고리: 가격 책정<br>감정: 부정<br>우선순위: 높음<br><br>4. 모바일 앱에서 대용량 파일(>50MB)을 업로드하려고 하면 계속 충돌이 일어나요. 이건 우리 현장 요원들에게 아주 중요해요.<br>카테고리: 성능<br>감정: 부정<br>우선순위: 높음<br><br>5. 전반적으로 제품에 만족해요. UI가 직관적이고 고객 지원도 최고예요. 계속 잘 해주세요!<br>카테고리: UI/UX, 기타 (고객 지원)<br>감정: 긍정<br>우선순위: 낮음 |

In [ ]:
FEEDBACK_DATA_2 = """
새 대시보드가 엉망이에요! 로딩하는 데 시간이 오래 걸리고 내보내기 버튼을 찾을 수 없어요. 당장 고쳐주세요!
"""

prompt_02 = f"""
이 고객 피드백을 분석하고 문제를 분류하세요. 다음 카테고리를 사용하세요: UI/UX, 성능, 기능 요청, 통합, 가격 책정, 기타. 또한 감정(긍정/중립/부정)과 우선순위(높음/중간/낮음)를 평가하세요.

피드백은 다음과 같습니다: {FEEDBACK_DATA_2}
"""

# prompt_02 = f"""
# 우리 CS 팀은 비정형 피드백으로 인해 압도되고 있습니다. 당신의 임무는 피드백을 분석하고 우리 제품 및 엔지니어링 팀을 위해 문제를 분류하는 것입니다. 다음 카테고리를 사용하세요: UI/UX, 성능, 기능 요청, 통합, 가격 책정, 기타. 또한 감정(긍정/중립/부정)과 우선순위(높음/중간/낮음)를 평가하세요. 다음은 예시입니다:

# <example>
# 입력: 새 대시보드가 엉망이에요! 로딩하는 데 시간이 오래 걸리고 내보내기 버튼을 찾을 수 없어요. 당장 고쳐주세요!
# 카테고리: UI/UX, 성능
# 감정: 부정
# 우선순위: 높음
# </example>

# 이제 이 피드백을 분석하세요: {FEEDBACK_DATA_2}
# """

answer_02 = invoke_claude(prompt_02)
print(f"\n{answer_02}\n")

## 3. Claude가 생각하게 하기(chain of thought prompting)로 성능 향상시키기

연구, 분석 또는 문제 해결과 같은 복잡한 작업에 직면했을 때, Claude에게 생각할 공간을 주는 것은 그의 성능을 크게 향상시킬 수 있습니다. 연쇄 사고(CoT) 프롬프팅으로 알려진 이 기술은 Claude가 문제를 단계별로 분해하도록 장려하여 더 정확하고 뉘앙스 있는 결과를 이끌어냅니다.

### Claude가 생각하게 하는 이유는 무엇인가요?

- 정확성: 문제를 단계별로 해결하면 특히 수학, 논리, 분석 또는 일반적으로 복잡한 작업에서 오류가 줄어듭니다.
- 일관성: 구조화된 사고는 더 응집력 있고 잘 조직된 응답으로 이어집니다.
- 디버깅: Claude의 사고 과정을 보면 프롬프트가 불명확할 수 있는 곳을 정확히 파악하는 데 도움이 됩니다.


In [ ]:
PROGRAM_DETAILS = "Care for Kids"
DONOR_DETAILS = "Andrew"

prompt_03 = f"""
올해 Care for Kids 프로그램에 기부를 요청하는 기부자 맞춤형 이메일을 작성하세요.

프로그램 정보:
<program>{PROGRAM_DETAILS}
</program>

기부자 정보:
<donor>{DONOR_DETAILS}
</donor>

이메일을 작성하기 전에 단계별로 생각하세요.
"""

# prompt_03 = f"""
# 올해 Care for Kids 프로그램에 기부를 요청하는 기부자 맞춤형 이메일을 작성하세요.

# 프로그램 정보:
# <program>{PROGRAM_DETAILS}
# </program>

# 기부자 정보:
# <donor>{DONOR_DETAILS}
# </donor>

# <thinking> 태그 안에서 이메일을 작성하기 전에 생각하세요. 
# 먼저 기부자의 기부 이력과 그들이 과거에 지원했던 캠페인을 고려할 때 어떤 메시지가 이 기부자에게 호소력이 있을지 생각해 보세요. 
# 그런 다음 기부자의 이력을 감안할 때 Care for Kids 프로그램의 어떤 측면이 그들에게 매력적일지 생각해 보세요. 
# 마지막으로 <email> 태그 안에 분석 내용을 활용하여 기부자 맞춤형 이메일을 작성하세요.
# """

answer_03 = invoke_claude(prompt_03, 4000)
print(f"\n{answer_03}\n")

## 4. 프롬프트 구조화를 위해 XML 태그 사용하기

프롬프트에 컨텍스트, 지침, 예시 등 여러 구성 요소가 포함되어 있다면 XML 태그가 게임 체인저가 될 수 있습니다. XML 태그는 Claude가 프롬프트를 더 정확하게 파싱하여 고품질 출력을 생성하는 데 도움이 됩니다.

### XML 태그를 사용하는 이유

- 명확성: 프롬프트의 각 부분을 명확하게 구분하고 프롬프트가 잘 구조화되도록 합니다.
- 정확성: 프롬프트의 일부를 Claude가 잘못 해석하여 발생하는 오류를 줄입니다.
- 유연성: 모든 것을 다시 작성하지 않고도 프롬프트의 특정 부분을 쉽게 찾고, 추가하고, 제거하거나 수정할 수 있습니다.
- 파싱 용이성: Claude가 출력에서 XML 태그를 사용하면 후처리를 통해 응답의 특정 부분을 더 쉽게 추출할 수 있습니다.

| 역할 | XML 태그 없음 | XML 태그 사용 |
|------|--------------|---------------|
| 사용자 | AcmeCorp의 재무 분석가입니다. 투자자를 위한 Q2 재무 보고서를 작성하세요. 작년 Q1 보고서 예시처럼 매출 성장, 이익률, 현금 흐름에 대한 섹션을 포함하세요: {{Q1_REPORT}}. 이 스프레드시트의 데이터 포인트를 사용하세요: {{SPREADSHEET_DATA}}. 보고서는 매우 간결하고 직접적이며 전문적이고 목록 형식이어야 합니다. 강점과 개선 영역을 모두 강조해야 합니다. | AcmeCorp의 재무 분석가입니다. 투자자를 위한 Q2 재무 보고서를 작성하세요.<br><br>AcmeCorp는 B2B SaaS 기업입니다. 투자자들은 투명성과 실행 가능한 인사이트를 중요하게 여깁니다.<br><br>보고서에 이 데이터를 사용하세요:`<data>`{{SPREADSHEET_DATA}}`</data>`<br><br>`<instructions>`<br>1. 포함할 섹션: 매출 성장, 이익률, 현금 흐름.<br>2. 강점과 개선 영역을 강조하세요.<br>`</instructions>`<br><br>간결하고 전문적인 톤을 사용하세요. 다음 구조를 따르세요:<br>`<formatting_example>`{{Q1_REPORT}}`</formatting_example>` |
| Assistant | AcmeCorp 투자자를 위한 Q2 재무 보고서<br><br>존경하는 투자자 여러분,<br><br>B2B SaaS 시장에서 AcmeCorp의 지속적인 성장과 회복력을 보여주는 Q2 재무 보고서를 기쁜 마음으로 제출합니다.<br><br>1. 매출 성장:<br>Q2 매출은 전년 동기 대비 22% 증가한 `$15.2M`에 달했습니다. 이 성장은 주로 기업 고객 기반이 30% 확대된 데 기인합니다. 그러나 SMB 부문은 5% 소폭 감소했는데, 이는 Q3에 타겟 마케팅 이니셔티브를 통해 해결하고자 합니다.<br><br>2. 이익률:<br>인프라 최적화로 인해 매출총이익률은 Q1 대비 3% 상승한 72%로 개선되었습니다. 그러나 EBITDA 마진은 예정된 AI 기반 기능에 대한 R&D 투자로 인해 2% 하락한 18%를 기록했습니다.<br><br>3. 현금 흐름:<br>영업 현금 흐름은 `$4.1M`으로 견고합니다. 현금 보유액은 `$28M`으로, 현재 소진율로는 15개월 분량입니다. 고객 확보 비용이 20% 증가했는데, 이는 더 효율적인 영업 전략을 통해 해결하고 있습니다.<br><br>요약하면, AcmeCorp의 핵심 재무는 견고합니다. 성장 투자와 신중한 현금 관리의 균형을 맞추고 있습니다. Q3의 초점은 SMB 성장 활성화와 영업 효율성 최적화가 될 것입니다.<br><br>지속적인 신뢰에 감사드립니다.<br><br>CFO 드림,<br>AcmeCorp | `<output>`AcmeCorp 투자자를 위한 Q2 재무 보고서<br><br>매출 성장:<br>- Q2 매출: `$15.2M` (전년 동기 대비 22% ↑)<br>- 기업 고객: 30% ↑<br>- SMB 부문: 5% ↓<br>조치: Q3 타겟 마케팅<br><br>이익률:<br>- 매출총이익률: 72% (Q1 대비 3% ↑)<br>이유: 인프라 최적화<br>- EBITDA 마진: 18% (2% ↓)<br>이유: AI 기능에 대한 R&D 투자<br><br>현금 흐름:<br>- 영업 현금 흐름: `$4.1M` (견고함)<br>- 현금 보유액: $28M (15개월 분량)<br>- 고객 확보 비용: 20% ↑<br>조치: 영업 전략 최적화<br><br>전망:<br>- 핵심 재무: 견고함<br>- Q3 초점: SMB 활성화, 영업 최적화<br><br>경영진은 성장 투자와 재무 규율의 균형을 맞추기 위해 최선을 다하고 있습니다.`</output>` |

In [ ]:
Q1_REPORT = """
AcmeCorp 투자자를 위한 Q1 재무 보고서

매출 성장:
- Q1 매출: $12.0M (전년 동기 대비 22% ↑)
- 기업 고객: 10% ↑
- SMB 부문: 2% ↓
조치: Q2 타겟 마케팅

이익률:
- 매출총이익률: 70% (Q4 대비 1% ↑)
이유: 인프라 최적화
- EBITDA 마진: 18.5% (1% ↓)
이유: AI 기능에 대한 R&D 투자

현금 흐름:
- 영업 현금 흐름: $4.1M (견고함)
- 현금 보유액: $28M (15개월 분량)
- 고객 확보 비용: 15% ↑
조치: 영업 전략 최적화

전망:
- 핵심 재무: 견고함
- Q2 초점: SMB 활성화, 영업 최적화

경영진은 성장 투자와 재무 규율의 균형을 맞추기 위해 최선을 다하고 있습니다.
"""

SPREADSHEET_DATA = """
분기,매출,기업고객,SMB부문,매출총이익률,EBITDA마진,영업 현금 흐름, 현금 보유액, 고객 확보 비용
Q2, $15.2M,4.8M,1.9M,72%,18%,$4.1M, $28M, 0.6M
"""

prompt_04 = f"""
AcmeCorp의 재무 분석가입니다. 투자자를 위한 Q2 재무 보고서를 작성하세요. 
작년 Q1 보고서 예시처럼 매출 성장, 이익률, 현금 흐름에 대한 섹션을 포함하세요: {Q1_REPORT}. 
이 스프레드시트의 데이터 포인트를 사용하세요: {SPREADSHEET_DATA}. 
보고서는 매우 간결하고 직접적이며 전문적이고 목록 형식이어야 합니다. 강점과 개선 영역을 모두 강조해야 합니다.
"""

# prompt_04 = f"""
# AcmeCorp의 재무 분석가입니다. 투자자를 위한 Q2 재무 보고서를 작성하세요.

# AcmeCorp는 B2B SaaS 기업입니다. 투자자들은 투명성과 실행 가능한 인사이트를 중요하게 여깁니다.

# 보고서에 이 데이터를 사용하세요:<data>{SPREADSHEET_DATA}</data>

# <instructions>
# 1. 포함할 섹션: 매출 성장, 이익률, 현금 흐름.
# 2. 강점과 개선 영역을 강조하세요.
# </instructions>

# 간결하고 전문적인 톤을 사용하세요. 다음 구조를 따르세요:
# <formatting_example>{Q1_REPORT}</formatting_example>
# """

answer_04 = invoke_claude(prompt_04, 4000)
print(f"\n{answer_04}\n")

## 5. 시스템 프롬프트로 Claude에게 역할 부여하기

Claude를 사용할 때, system 파라미터를 사용하여 역할을 부여함으로써 성능을 크게 향상시킬 수 있습니다. 역할 프롬프팅으로 알려진 이 기술은 Claude와 함께 시스템 프롬프트를 사용하는 가장 강력한 방법입니다.

적절한 역할은 Claude를 일반 어시스턴트에서 가상 도메인 전문가로 변모시킬 수 있습니다!

### 왜 역할 프롬프팅을 사용하나요?

- 정확도 향상: 법률 분석이나 재무 모델링과 같은 복잡한 시나리오에서 역할 프롬프팅은 Claude의 성능을 크게 향상시킬 수 있습니다.
- 맞춤형 톤: CFO의 간결함이나 카피라이터의 재치가 필요하든, 역할 프롬프팅은 Claude의 의사소통 스타일을 조정합니다.
- 집중력 향상: 역할 컨텍스트를 설정함으로써 Claude는 작업의 특정 요구사항 내에서 더 머물 수 있습니다.

**역할 프롬프팅 없는 재무 분석**

| 역할 | 내용 |
|------|------|
| User | 2분기 재무 데이터셋을 분석하세요: <br>`<data>` <br>{{FINANCIALS}} <br>`</data>` <br><br>주요 추세를 강조하고 조치를 권장하세요. |
| Assistant | 2분기 재무 현황을 요약하면 다음과 같습니다: <br><br>1. 매출: `$15.2M`, 전년 대비 22% 증가. 기업 고객 기반 30% 성장. <br>2. 매출총이익률: 72%, 1분기 대비 3% 증가. <br>3. EBITDA 마진: 18%, R&D 투자로 인해 2% 감소. <br>4. 현금 흐름: `$4.1M` 영업, `$28M` 보유. <br><br>전반적으로 매출 성장과 건전한 마진으로 재무 상태가 견고해 보입니다. R&D 지출 증가가 수익성에 영향을 미치고 있지만 장기적으로 성과를 낼 수 있습니다. 현재 방향을 유지하세요. |

**역할 프롬프팅 있는 재무 분석**

| 역할 | 내용 |
|------|------|
| User | 당신은 고성장 B2B SaaS 기업의 CFO입니다. 우리는 이사회에서 2분기 재무 현황을 논의하고 있습니다: <br>`<data>` <br>{{FINANCIALS}} <br>`</data>` <br><br>주요 추세를 분석하고, 우려 사항을 지적하며, 전략적 조치를 권고하세요. 우리 투자자들은 공격적인 성장을 원하지만 우리의 소진율을 경계하고 있습니다. |
| Assistant | 이사회 여러분, 2분기 재무 현황에 대한 제 분석은 다음과 같습니다: <br><br>1. 매출(`$15.2M`, 전년 대비 +22%): <br>- 통찰: 기업 성장(+30%)이 전체 성장을 앞지르고 있습니다. <br>- 주의: 중소기업 부문이 5% 감소했습니다. <br>- 조치: 중소기업 마케팅 예산의 20%를 기업 영업으로 재배분하세요. 제품팀에 중소기업 온보딩 단순화를 지시하세요. <br><br>2. 마진(매출총이익률: 72%, 전분기 대비 +3%; EBITDA: 18%, 전분기 대비 -2%): <br>- 통찰: 인프라 최적화로 매출총이익률이 상승했습니다. <br>- 주의: AI 기능에 대한 R&D 지출이 EBITDA를 떨어뜨렸습니다. <br>- 조치: 변경 없음. 우리의 AI 로드맵은 기업 고착화에 매우 중요합니다. <br><br>3. 현금(`$4.1M` 영업 현금 흐름, $28M 보유): <br>- 통찰: 현재 소진율로 15개월 운영 가능. <br>- 주의: 고객 획득 비용 20% 증가, LTV/CAC 악화. <br>- 조치: 필수 인력 외 채용 동결. 3분기에 고객 획득 비용을 15% 줄이기 위해 영업 퍼널 분석을 실시하세요. <br><br>전략적 입장: 우리는 성장과 소진 사이에서 균형을 잡고 있습니다. 기업 중심이 우리의 성장 동력입니다. 거기에 투자하세요. 그러나 중소기업 이탈을 주시하세요. 그것이 기업으로 번질 수 있습니다. 우리의 AI 투자는 건전하지만, 4분기까지 연간 반복 매출로 전환해야 합니다. 그렇지 않으면 R&D를 30% 삭감합니다. <br><br>요약하면: 기업에 집중하고, 중소기업을 간소화하며, 영업을 최적화하고, AI를 수익화하세요. 질문 있으신가요? |


In [ ]:
FINANCIALS = """
고성장 B2B SaaS 기업 2분기 재무 보고서

1. 경영진 요약
   당사는 2분기에 강력한 성장세를 유지하며 전년 대비 22%의 매출 증가를 기록했습니다. 
   기업 고객 부문의 성장이 두드러졌으나, 중소기업(SMB) 부문의 성과 저하가 관찰되었습니다. 
   수익성 지표는 혼조세를 보이며, 현금 관리에 주의를 기울일 필요가 있습니다.

2. 재무 성과 개요
   2.1 손익계산서 주요 항목
       - 총매출: $15.2M (전년 대비 22% 증가)
       - 매출원가: $4.26M (매출의 28%)
       - 매출총이익: $10.94M (매출총이익률 72%, 전분기 대비 3%p 상승)
       - 영업비용: $8.18M (매출의 53.8%)
       - EBITDA: $2.74M (EBITDA 마진 18%, 전분기 대비 2%p 하락)
       - 순이익: $1.52M (순이익률 10%)

   2.2 대차대조표 주요 항목
       - 현금 및 현금성자산: $28M
       - 매출채권: $3.8M
       - 총자산: $45M
       - 총부채: $15M
       - 자기자본: $30M

   2.3 현금흐름표 주요 항목
       - 영업활동 현금흐름: $4.1M
       - 투자활동 현금흐름: -$2.5M (주로 R&D 및 설비투자)
       - 재무활동 현금흐름: $0M
       - 순현금증감: $1.6M

3. 부문별 성과 분석
   3.1 기업 고객 부문
       - 매출: $10.64M (전체 매출의 70%, 전년 대비 30% 성장)
       - 고객 수: 250개 (순증 50개)
       - 평균 고객당 매출(ARPU): $42,560

   3.2 중소기업(SMB) 부문
       - 매출: $4.56M (전체 매출의 30%, 전년 대비 5% 감소)
       - 고객 수: 2,280개 (순감 120개)
       - 평균 고객당 매출(ARPU): $2,000

   3.3 제품별 성과
       - 핵심 SaaS 플랫폼: $12.16M (전체 매출의 80%)
       - AI 기반 부가 서비스: $3.04M (전체 매출의 20%, 전년 대비 50% 성장)

4. 주요 재무 지표 및 KPI
   - 고객 획득 비용(CAC): $5,000 (전분기 대비 20% 증가)
   - 고객 생애 가치(LTV): $25,000
   - LTV/CAC 비율: 5:1 (업계 권장 3:1 이상)
   - 연간반복수익(ARR): $60.8M (전년 대비 22% 증가)
   - 순수익유지율(NRR): 110%
   - 이탈률: 5% (연간)

5. 전략적 이니셔티브 및 향후 전망
   5.1 기업 고객 중심 전략 강화
       - 기업 영업팀 20% 확대
       - 대기업 맞춤형 솔루션 개발 가속화

   5.2 SMB 부문 효율화
       - 온보딩 프로세스 간소화로 고객 경험 개선
       - 자동화된 고객 지원 시스템 도입으로 운영 효율성 제고

   5.3 AI 기술 투자 및 수익화
       - 4분기까지 AI 기반 서비스의 ARR 기여도 20% 달성 목표
       - 신규 AI 기능 3개 출시 예정 (3분기 중)

   5.4 운영 효율성 개선
       - 영업 프로세스 최적화로 CAC 15% 감소 목표 (3분기 중)
       - 필수 인력 외 채용 동결로 인건비 관리

6. 리스크 요인 및 완화 전략
   6.1 SMB 고객 이탈 리스크
       - 맞춤형 유지 프로그램 도입
       - 제품 사용성 개선 태스크포스 운영

   6.2 기술 투자 ROI 리스크
       - AI 투자 성과 월별 모니터링 체계 구축
       - 4분기 목표 미달성 시 R&D 예산 30% 삭감 계획 수립

   6.3 경쟁 심화 리스크
       - 특허 출원 가속화 (2분기 중 5건 출원)
       - 핵심 기술 인력 유지를 위한 인센티브 프로그램 강화

7. 결론 및 전망
   당사는 견고한 성장세를 유지하고 있으며, 특히 기업 고객 부문에서의 성과가 
   뛰어납니다. 단기적으로는 SMB 부문의 안정화와 운영 효율성 개선에 주력하겠습니다. 
   중장기적으로는 AI 기술 투자를 통한 경쟁우위 확보와 지속 가능한 고성장에 
   집중하겠습니다. 현재의 재무 상태와 시장 포지션을 고려할 때, 연간 목표 달성이 
   가능할 것으로 전망됩니다.

   다음 분기에는 신규 기업 고객 유치, AI 서비스의 수익화, 그리고 운영 효율성 
   제고를 통한 수익성 개선에 모든 역량을 집중하겠습니다.
"""

prompt_05 = f"""
	2분기 재무 데이터셋을 분석하세요:
<data>
{FINANCIALS}
</data>

주요 추세를 강조하고 조치를 권장하세요.
"""

# prompt_05 = f"""
# 당신은 고성장 B2B SaaS 기업의 CFO입니다. 우리는 이사회에서 2분기 재무 현황을 논의하고 있습니다:
# <data>
# {FINANCIALS}
# </data>

# 주요 추세를 분석하고, 우려 사항을 지적하며, 전략적 조치를 권고하세요. 우리 투자자들은 공격적인 성장을 원하지만 우리의 소진율을 경계하고 있습니다.
# """

answer_05 = invoke_claude(prompt_05, 4000)
print(f"\n{answer_05}\n")

## 6. 출력 제어를 위해 Claude의 응답 미리 채우기

Claude를 사용할 때, Assistant 메시지를 미리 채워 응답을 유도할 수 있는 독특한 기능이 있습니다. 이 강력한 기술을 통해 Claude의 행동을 지시하고, 서문을 건너뛰고, JSON이나 XML과 같은 특정 형식을 적용하며, 롤플레이 시나리오에서 캐릭터의 일관성을 유지하는 데 도움을 줄 수 있습니다.

Claude가 기대한 대로 수행되지 않는 경우, 몇 문장을 미리 채우는 것만으로도 Claude의 성능을 크게 향상시킬 수 있습니다. 약간의 사전 입력으로 큰 효과를 볼 수 있습니다!

### Claude의 응답을 미리 채우는 방법

미리 채우려면 Assistant 메시지에 원하는 초기 텍스트를 포함시키세요(Claude의 응답은 Assistant 메시지가 끝나는 지점부터 이어집니다):

```python
import anthropic

client = anthropic.Anthropic()
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "당신이 가장 좋아하는 색깔은 무엇인가요?"},
        {"role": "assistant", "content": "AI 어시스턴트로서 저는 좋아하는 색깔이 없습니다. 하지만 굳이 고르자면 초록색일 것 같은데, 그 이유는"}  # 여기에 미리 채우기
    ]
)
```

### 예시

**미리 채우기 없이 구조화된 데이터 추출**

| 역할 | 내용 |
|------|------|
| 사용자 | 다음 제품 설명에서 이름, 크기, 가격, 색상을 JSON 객체로 추출해주세요: <br><br>`<description>`<br>SmartHome Mini는 검정색 또는 흰색으로 제공되는 소형 스마트 홈 어시스턴트로, 가격은 단돈 `$49.99`입니다. 너비가 5인치에 불과하지만, 집 어디에 두더라도 음성이나 앱을 통해 조명, 온도 조절기 및 기타 연결된 기기를 제어할 수 있습니다. 이 저렴한 작은 허브는 스마트 기기에 편리한 핸즈프리 제어 기능을 제공합니다.<br>`</description>` |
| 어시스턴트 | 추출한 정보를 JSON 형식으로 제공하겠습니다: <br><br>\`\`\`json<br>{ <br>`"name"`: `"SmartHome Mini"`,<br>`"size"`: `"너비 5인치"`,<br>`"price"`: `"$49.99"`,<br>`"colors"`: [`"검정색"`, `"흰색"`]<br>}<br>\`\`\`<br><br>제품 설명에서 다음 세부 정보를 추출했습니다:<br><br>- **이름**: SmartHome Mini<br>- **크기**: 너비 5인치<br>- **가격**: $49.99<br>- **색상**: 검정색과 흰색으로 제공<br><br>JSON 객체는 프로그램에서 쉽게 파싱하고 사용할 수 있는 구조화된 형식을 제공합니다. 수정이 필요하거나 다른 질문이 있으시면 알려주세요! |

**미리 채우기로 구조화된 데이터 추출**

| 역할 | 내용 |
|------|------|
| 사용자 | 다음 제품 설명에서 이름, 크기, 가격, 색상을 JSON 객체로 추출해주세요: <br><br>`<description>`<br>SmartHome Mini는 검정색 또는 흰색으로 제공되는 소형 스마트 홈 어시스턴트로, 가격은 단돈 `$49.99`입니다. 너비가 5인치에 불과하지만, 집 어디에 두더라도 음성이나 앱을 통해 조명, 온도 조절기 및 기타 연결된 기기를 제어할 수 있습니다. 이 저렴한 작은 허브는 스마트 기기에 편리한 핸즈프리 제어 기능을 제공합니다.<br>`</description>` |
| 어시스턴트 (미리 채우기) | { |
| 어시스턴트 (Claude의 응답) | `"name"`: `"SmartHome Mini"`,<br>`"size"`: `"너비 5인치"`,<br>`"price"`: `"$49.99"`,<br>`"colors"`: [<br>  `"검정색"`,<br>  `"흰색"`<br>]<br>} |

## 한 가지 더: 프롬프트 엔지니어링 실전 팁

### 성공 기준 정의하기

성공적인 LLM 기반 애플리케이션을 구축하려면 성공 기준을 명확히 정의하는 것부터 시작해야 합니다. 애플리케이션을 출시할 만큼 충분히 좋아졌다는 것을 어떻게 알 수 있을까요?

명확한 성공 기준이 있으면 프롬프트 엔지니어링 및 최적화 노력이 구체적이고 측정 가능한 목표 달성에 집중될 수 있습니다.

**기준 예시**

- 작업 충실도
- 일관성
- 관련성 및 일치성
- 톤과 스타일
- 개인정보 보호
- 컨텍스트 활용
- 지연시간
- 비용

### 평가 cycle 구축하기

성공 기준을 정의했으면 그 다음 단계는 평가를 구축하는 것입니다. 이는 프롬프트 엔지니어링 사이클의 중요한 부분입니다.

![process](https://mintlify.s3-us-west-1.amazonaws.com/anthropic/images/how-to-prompt-eng.png)

평가를 설계할 때는 다음과 같은 점을 주의해야 합니다.

1. **작업 기준**: 실제 작업을 반영하는 평가를 설계합니다. 이 때 다양한 edge case를 고려합니다.
2. **가능한 자동화**: 질문을 구조화해서 자동 채점이 가능하도록 합니다.
3. **품질보다 양 우선**: 약간 낮은 품질의 자동 채점으로 많은 양을 수행하는 것이 고품질의 수동 채점으로 더 적은 질문을 하는 것보다 낫습니다.